In [52]:
import pandas as pd


file_path = "C:\\Users\\abubabu\\Documents\\GitHub\\synthetic-population-simulation\\NHTS Data Parser\\HH Data Parser\\trippub.csv"
# df = pd.read_csv(file_path)

columns_to_keep = ['HOUSEID', 'PERSONID',"STRTTIME","ENDTIME",'TRVLCMIN', 'R_SEX', 'R_AGE', 'WRKCOUNT','HHVEHCNT', 'HHSIZE','EDUC','WHYTRP1S','HHFAMINC']
final_data = pd.read_csv(file_path,usecols=columns_to_keep)




In [53]:
def translateAge(age):
    if age <=5:
        return 1
    elif age <= 18:
        return 2
    elif age <= 25:
        return 3
    elif age < 60:
        return 4
    else:
        return 5

# get the symbolic name of the gender from numerical value
def translateSex(sex):
    if sex == 1:
        return 'male'
    else:
        return 'female'

# get the race categories from numerical race values
def translateRace(race):
    if race == 1:
        return 1 #white
    elif race == 2:
        return 2 #black
    elif race == 4 and race ==6:
        return 3 #native
    elif race == 3:
        return 6 #asian
    elif race == 5:
        return 7 #'pacific-islander'
    else:
        return 8 #'other'
    
def translateInc(inc):
    if inc>=10: return 10
    if inc<1: return 0
    return inc   

def translateTSum(ts):
    if ts==1: return 1 #Home
    elif ts==10: return 2 #work
    elif ts==20: return 3 #school
    elif ts==30: return 4 #medical
    elif ts==40: return 5 #shopping
    elif ts==50: return 6 #social
    elif ts==70: return 7 #transport
    elif ts==80: return 8 #meals
    elif ts==97: return 9 #other

In [54]:
import datetime
final_data['R_AGE'] = final_data.apply(lambda row: translateAge(row['R_AGE']), axis=1)
# final_data['R_RACE'] = final_data.apply(lambda row: translateRace(row['R_RACE']), axis=1)
final_data['HHFAMINC'] = final_data.apply(lambda row: translateInc(row['HHFAMINC']), axis=1)
final_data['WHYTRP1S'] = final_data.apply(lambda row: translateTSum(row['WHYTRP1S']), axis=1)
final_data['STRTTIME'] = final_data['STRTTIME'].apply(lambda x: datetime.time(hour=x // 100, minute=x % 100))
final_data['ENDTIME'] = final_data['ENDTIME'].apply(lambda x: datetime.time(hour=x // 100, minute=x % 100))
# final_data['TRVLCMIN'] = final_data['TRVLCMIN'].apply(lambda x: datetime.time(hour=x // 100, minute=x % 100))



In [57]:
final_data['STRTTIME'].value_counts()

17:00:00    17077
16:00:00    15211
15:00:00    14534
10:00:00    14212
11:00:00    13626
            ...  
04:39:00        1
02:26:00        1
02:21:00        1
03:46:00        1
03:41:00        1
Name: STRTTIME, Length: 1397, dtype: int64

In [56]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf

X = final_data.drop(columns=['WHYTRP1S','HOUSEID', 'PERSONID'],axis=1)
y = final_data['WHYTRP1S']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
print(X_train_scaled.shape)
print(y_train.shape)
print(X.shape)
num_classes =10
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)

TypeError: float() argument must be a string or a real number, not 'datetime.time'

In [25]:
# Create the neural network model
model = Sequential()
model.add(Dense(64, input_dim=X_train_scaled.shape[1], activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(X_train_scaled, y_train, epochs=3, batch_size=32, validation_data=(X_test_scaled, y_test))
# model.fit(X_train, y, epochs=3, batch_size=32, validation_split=0.2)

# Evaluate the model
loss, accuracy = model.evaluate(X_test_scaled, y_test)
print("Test loss:", loss)
print("Test accuracy:", accuracy)

Epoch 1/3
23090/23090 [==============================] - 17s 702us/step - loss: 1.5976 - accuracy: 0.4399 - val_loss: 1.5749 - val_accuracy: 0.4456
Epoch 2/3
23090/23090 [==============================] - 16s 693us/step - loss: 1.5735 - accuracy: 0.4454 - val_loss: 1.5685 - val_accuracy: 0.4469
Epoch 3/3
5773/5773 [==============================] - 3s 495us/step - loss: 1.5633 - accuracy: 0.4483
Test loss: 1.5633224248886108
Test accuracy: 0.44831767678260803


In [46]:
model.save("C:\\Users\\abubabu\\Documents\\GitHub\\synthetic-population-simulation\\Model\\v3_oh.h5")

In [47]:
data_to_predict=pd.read_csv("C:\\Users\\abubabu\\Documents\\GitHub\\synthetic-population-simulation\Model\\shundordata.csv")
data_to_predict=data_to_predict.drop(['Unnamed: 0'],axis=1)
data_to_predict.head(5)

,STRTTIME,ENDTIME,TRVLCMIN,HHSIZE,HHVEHCNT,HHFAMINC,WRKCOUNT,R_AGE,EDUC,R_SEX
0,1000,1100,60,3,0.0,10,0.0,5,3,2
1,1000,1100,60,3,0.0,10,0.0,5,3,2
2,1000,1100,60,3,0.0,10,0.0,5,3,2
3,1000,1100,60,2,3.0,10,3.0,4,1,2
4,1000,1100,60,2,3.0,10,3.0,4,1,2


In [48]:
data_to_predict = scaler.transform(data_to_predict)
predictions = model.predict(X_test)
predicted_classes = np.argmax(predictions, axis=1)

5773/5773 [==============================] - 3s 480us/step


In [45]:
# Specify the filename for the CSV file
filename = 'data.csv'

# Save the NumPy array to the CSV file
np.savetxt(filename, predictions, delimiter=',')

In [49]:
predictions

array([[0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       ...,
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.]], dtype=float32)

In [82]:
final_data['HHFAMINC'].value_counts()

6     160854
10    138405
7     132897
8     110961
5     100756
4      70939
9      64388
3      59997
2      30177
1      29858
0      24340
Name: HHFAMINC, dtype: int64

In [83]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.preprocessing import MinMaxScaler

# Assuming "final_data" is a pandas DataFrame containing your dataset
# Make sure you have already preprocessed the data and removed any missing values.

# Extracting the features (X) and target (y) from the dataset
# X = final_data.drop(columns=['TRIPPURP','WHYFROM','HOUSEID', 'PERSONID',"TDAYDATE",'WRKCOUNT'],axis=1)
X = final_data.drop(columns=['WHYTRP1S','HOUSEID', 'PERSONID'],axis=1)
y = final_data['WHYTRP1S']

# Preprocess 'R_AGE' and 'R_SEX' using LabelEncoder
# label_encoder = LabelEncoder()
# X['R_AGE'] = label_encoder.fit_transform(X['R_AGE'])
# X['R_SEX'] = label_encoder.fit_transform(X['R_SEX'])


# Preprocess the target variable using LabelEncoder
y_label_encoder = LabelEncoder()
y_encoded = y_label_encoder.fit_transform(y)

# Convert the target variable to one-hot encoded format
num_classes = len(y_label_encoder.classes_)
y_one_hot = pd.get_dummies(y_encoded)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y_one_hot, test_size=0.2, random_state=42)

# X_train=X.astype('float64')
num_classes
# Standardize the features (optional, but often recommended for neural networks)
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
input_shape = X_train.shape[1]

# X_test_scaled.shape
print(X_train_scaled.shape)
print(y_train.shape)
print(X.shape)
final_data['WHYTRP1S'].value_counts()


(738857, 10)
(738857, 9)
(923572, 10)


1     318777
40    184126
10    110590
50    100284
80     72327
70     56377
20     43397
97     20910
30     16784
Name: WHYTRP1S, dtype: int64

In [84]:
original_classes = y_label_encoder.classes_

# Print the original class labels and their corresponding one-hot encoded vectors
for i, class_label in enumerate(original_classes):
    one_hot_vector = y_one_hot.iloc[i]  # Get the i-th row of the one-hot encoded DataFrame
    print(f"Class Label: {class_label}, One-Hot Vector: {one_hot_vector.to_list()}")

Class Label: 1, One-Hot Vector: [0, 0, 1, 0, 0, 0, 0, 0, 0]
Class Label: 10, One-Hot Vector: [1, 0, 0, 0, 0, 0, 0, 0, 0]
Class Label: 20, One-Hot Vector: [1, 0, 0, 0, 0, 0, 0, 0, 0]
Class Label: 30, One-Hot Vector: [0, 1, 0, 0, 0, 0, 0, 0, 0]
Class Label: 40, One-Hot Vector: [0, 0, 1, 0, 0, 0, 0, 0, 0]
Class Label: 50, One-Hot Vector: [1, 0, 0, 0, 0, 0, 0, 0, 0]
Class Label: 70, One-Hot Vector: [0, 1, 0, 0, 0, 0, 0, 0, 0]
Class Label: 80, One-Hot Vector: [1, 0, 0, 0, 0, 0, 0, 0, 0]
Class Label: 97, One-Hot Vector: [0, 0, 0, 0, 0, 1, 0, 0, 0]


In [85]:
y_label_encoder.classes_

array([ 1, 10, 20, 30, 40, 50, 70, 80, 97], dtype=int64)

In [86]:
X['STRTTIME'].dtypes

dtype('int64')

In [87]:
y_one_hot.value_counts()

0  1  2  3  4  5  6  7  8
1  0  0  0  0  0  0  0  0    318777
0  0  0  0  1  0  0  0  0    184126
   1  0  0  0  0  0  0  0    110590
   0  0  0  0  1  0  0  0    100284
               0  0  1  0     72327
                  1  0  0     56377
      1  0  0  0  0  0  0     43397
      0  0  0  0  0  0  1     20910
         1  0  0  0  0  0     16784
dtype: int64

In [88]:
X_train_scaled

array([[0.36286562, 0.3819415 , 0.01157982, ..., 0.75      , 0.76923077,
        1.        ],
       [0.69944892, 0.70156846, 0.01157982, ..., 1.        , 0.92307692,
        1.        ],
       [0.50869012, 0.51292921, 0.01571547, ..., 0.25      , 0.53846154,
        1.        ],
       ...,
       [0.29673591, 0.29885545, 0.01157982, ..., 0.75      , 0.76923077,
        1.        ],
       [0.86689275, 0.86901229, 0.01157982, ..., 0.75      , 1.        ,
        0.66666667],
       [0.89020772, 0.89656634, 0.01985112, ..., 0.5       , 0.92307692,
        1.        ]])

In [71]:


# Create the neural network model
model = Sequential()
model.add(Dense(64, input_dim=X_train_scaled.shape[1], activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))
# model = Sequential()

# # Add layers to the model
# model.add(Dense(64, activation='relu', input_shape=(input_shape,)))
# model.add(Dense(32, activation='relu'))
# model.add(Dense(1))

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(X_train_scaled, y_train, epochs=3, batch_size=32, validation_data=(X_test_scaled, y_test))
# model.fit(X_train, y, epochs=3, batch_size=32, validation_split=0.2)

# Evaluate the model
loss, accuracy = model.evaluate(X_test_scaled, y_test)
print("Test loss:", loss)
print("Test accuracy:", accuracy)

Epoch 1/3
15829/23090 [===================>..........] - ETA: 4s - loss: 1.6290 - accuracy: 0.4316

KeyboardInterrupt: 

In [ ]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 64)                576       
                                                                 
 dense_4 (Dense)             (None, 32)                2080      
                                                                 
 dense_5 (Dense)             (None, 6)                 198       
                                                                 
Total params: 2,854
Trainable params: 2,854
Non-trainable params: 0
_________________________________________________________________


In [ ]:
X_train_scaled.shape

(923572, 8)

In [ ]:
model.save("C:\\Users\\abubabu\\Documents\\GitHub\\synthetic-population-simulation\\Model\\v1.h5")

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report

# Assuming you have X_train, X_test, y_train, y_test from the train-test split

# Create the Decision Tree classifier
dt_classifier = DecisionTreeClassifier(random_state=42)

# Train the model on the training data
dt_classifier.fit(X_train_scaled, y_train)

# Make predictions on the test data
y_pred = dt_classifier.predict(X_test_scaled)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# Print classification report
print("Classification Report:")
print(classification_report(y_test, y_pred))

Accuracy: 0.25870665620009203
Classification Report:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         5
           1       0.36      0.26      0.30     38092
           2       0.29      0.19      0.23     38881
           3       0.23      0.15      0.18     21993
           4       0.36      0.28      0.31     23445
           5       0.43      0.33      0.37     62299

   micro avg       0.36      0.26      0.30    184715
   macro avg       0.28      0.20      0.23    184715
weighted avg       0.35      0.26      0.30    184715
 samples avg       0.26      0.26      0.26    184715



C:\Users\abubabu\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [89]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam
from keras.callbacks import ReduceLROnPlateau

# Assuming "final_data" is a pandas DataFrame containing your dataset
# Make sure you have already preprocessed the data and removed any missing values.

# Extracting the features (X) and target (y) from the dataset
# X = final_data.drop(columns=['TRIPPURP', 'WHYFROM'])
# y = final_data['WHYFROM']

# # Preprocess 'R_AGE' and 'R_SEX' using LabelEncoder
# label_encoder = LabelEncoder()
# X['R_AGE'] = label_encoder.fit_transform(X['R_AGE'])
# X['R_SEX'] = label_encoder.fit_transform(X['R_SEX'])

# # Preprocess the target variable using LabelEncoder
# y_label_encoder = LabelEncoder()
# y_encoded = y_label_encoder.fit_transform(y)

# # Convert the target variable to one-hot encoded format
# num_classes = len(y_label_encoder.classes_)
# y_one_hot = pd.get_dummies(y_encoded)

# # Split the data into training and testing sets
# X_train, X_test, y_train, y_test = train_test_split(X, y_one_hot, test_size=0.2, random_state=42)

# # Standardize the features (optional, but often recommended for neural networks)
# scaler = StandardScaler()
# X_train_scaled = scaler.fit_transform(X_train)
# X_test_scaled = scaler.transform(X_test)

# Create the neural network model
model = Sequential()
model.add(Dense(128, input_dim=X_train_scaled.shape[1], activation='relu'))
model.add(Dropout(0.5))  # Adding dropout for regularization
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.5))  # Adding dropout for regularization
model.add(Dense(num_classes, activation='softmax'))

# Compile the model
optimizer = Adam(learning_rate=0.001)  # You can experiment with different learning rates
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

# Adding learning rate scheduler to reduce learning rate during training
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=0.0001)

# Train the model
model.fit(X_train_scaled, y_train, epochs=5, batch_size=32, validation_data=(X_test_scaled, y_test), callbacks=[reduce_lr])

# Evaluate the model
loss, accuracy = model.evaluate(X_test_scaled, y_test)
print("Test loss:", loss)
print("Test accuracy:", accuracy)

Epoch 1/5
23090/23090 [==============================] - 24s 1ms/step - loss: 1.6818 - accuracy: 0.4162 - val_loss: 1.6249 - val_accuracy: 0.4375 - lr: 0.0010
Epoch 2/5
23090/23090 [==============================] - 23s 1ms/step - loss: 1.6522 - accuracy: 0.4248 - val_loss: 1.6071 - val_accuracy: 0.4357 - lr: 0.0010
Epoch 3/5
23090/23090 [==============================] - 24s 1ms/step - loss: 1.6449 - accuracy: 0.4271 - val_loss: 1.6052 - val_accuracy: 0.4394 - lr: 0.0010
Epoch 4/5
23090/23090 [==============================] - 24s 1ms/step - loss: 1.6406 - accuracy: 0.4281 - val_loss: 1.5990 - val_accuracy: 0.4415 - lr: 0.0010
Epoch 5/5
5773/5773 [==============================] - 3s 525us/step - loss: 1.6012 - accuracy: 0.4419
Test loss: 1.6012290716171265
Test accuracy: 0.44188615679740906


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_squared_error, r2_score
model = LogisticRegression()
model.fit(X_train_scaled, y_train)
# Make predictions on the test set
y_pred = model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error: {mse}")
print(f"R-squared: {r2}")

ValueError: y should be a 1d array, got an array of shape (738857, 6) instead.

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization
from keras.optimizers import SGD
from keras.callbacks import LearningRateScheduler

# # Assuming "final_data" is a pandas DataFrame containing your dataset
# # Make sure you have already preprocessed the data and removed any missing values.

# # Extracting the features (X) and target (y) from the dataset
# X = final_data.drop(columns=['TRIPPURP', 'WHYFROM'])
# y = final_data['WHYFROM']

# # Preprocess 'R_AGE' and 'R_SEX' using LabelEncoder
# label_encoder = LabelEncoder()
# X['R_AGE'] = label_encoder.fit_transform(X['R_AGE'])
# X['R_SEX'] = label_encoder.fit_transform(X['R_SEX'])

# # Preprocess the target variable using LabelEncoder
# y_label_encoder = LabelEncoder()
# y_encoded = y_label_encoder.fit_transform(y)

# # Convert the target variable to one-hot encoded format
# num_classes = len(y_label_encoder.classes_)
# y_one_hot = pd.get_dummies(y_encoded)

# # Split the data into training and testing sets
# X_train, X_test, y_train, y_test = train_test_split(X, y_one_hot, test_size=0.2, random_state=42)

# # Standardize the features (optional, but often recommended for neural networks)
# scaler = StandardScaler()
# X_train_scaled = scaler.fit_transform(X_train)
# X_test_scaled = scaler.transform(X_test)

# Create the neural network model
model = Sequential()
model.add(Dense(128, input_dim=X_train_scaled.shape[1], activation='relu'))
model.add(BatchNormalization())  # Batch Normalization
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(BatchNormalization())  # Batch Normalization
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

# Define a learning rate scheduler
def lr_scheduler(epoch):
    return 0.001 * np.exp(-epoch / 10)

# Compile the model with SGD optimizer and learning rate scheduler
optimizer = SGD(learning_rate=0.0)  # Use a placeholder learning rate, the scheduler will update it
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

# Set up learning rate scheduler
lr_scheduler_callback = LearningRateScheduler(lr_scheduler)

# Train the model
model.fit(X_train_scaled, y_train, epochs=10, batch_size=32, validation_data=(X_test_scaled, y_test), callbacks=[lr_scheduler_callback])

# Evaluate the model
loss, accuracy = model.evaluate(X_test_scaled, y_test)
print("Test loss:", loss)
print("Test accuracy:", accuracy)

Epoch 1/10
23090/23090 [==============================] - 28s 1ms/step - loss: 1.8700 - accuracy: 0.3617 - val_loss: 1.6723 - val_accuracy: 0.4276 - lr: 0.0010
Epoch 2/10
23090/23090 [==============================] - 28s 1ms/step - loss: 1.7341 - accuracy: 0.3992 - val_loss: 1.6517 - val_accuracy: 0.4265 - lr: 9.0484e-04
Epoch 3/10
23090/23090 [==============================] - 28s 1ms/step - loss: 1.7137 - accuracy: 0.4037 - val_loss: 1.6431 - val_accuracy: 0.4260 - lr: 8.1873e-04
Epoch 4/10
23090/23090 [==============================] - 27s 1ms/step - loss: 1.7028 - accuracy: 0.4066 - val_loss: 1.6377 - val_accuracy: 0.4284 - lr: 7.4082e-04
Epoch 5/10
23090/23090 [==============================] - 27s 1ms/step - loss: 1.6950 - accuracy: 0.4094 - val_loss: 1.6338 - val_accuracy: 0.4292 - lr: 6.7032e-04
Epoch 6/10
23090/23090 [==============================] - 28s 1ms/step - loss: 1.6917 - accuracy: 0.4102 - val_loss: 1.6309 - val_accuracy: 0.4297 - lr: 6.0653e-04
Epoch 7/10
23090/230

In [ ]:
model.save("C:\\Users\\abubabu\\Documents\\GitHub\\synthetic-population-simulation\\Model\\v2.h5")